In [2]:
from scripts import pesquisa_ncbi
gene = 'hla-dqa2'
email = 'pg21019@alunos.uminho.pt'
pesquisa_ncbi(email,gene,db='nuccore',retmax=1,rettype='fasta',display='n',save='y')

Encontrados 99 resultados em nuccore. Irá ser processado 1 resultado.



'>OR557191.1 Homo sapiens isolate PSCDA0402 MHC class II antigen (HLA-DQA2) gene, HLA-DQA2*01:05:01:01extended allele, complete cds\nTTCAGATTTGCTTGTCTCGAGGTCCTCACAATTGCTCTACAGCTCAGAGCAGCAACTGCTGAGGCTGCCT\nTGGGAAGAAGATGATCCTAAACAAAGCTCTGCTGCTGGGGGCCCTCGCCCTGACTGCCGTGATGAGCCCC\nTGTGGAGGTGAAGACATTGTGGGTGAGTGCATGAGTGAGGAATGTTCTCTGGAGCTGAAAAACAGCAAAT\nTAAAGGAAAAGAAAGAGTGCAATTTGCTAAGAAATAGTAGAAATTTCCCAAGGGTCTTTTCAATATTAAG\nAAATTTTAAAATTATGGCAGTTCCTCCTTTAGGAAACCAGAGCTCCAACCGACTCTCTTTGCTACCTGTG\nCTATTGGAGTTTACCAAGGACGTTGTTCTGTTTATATTATATCCAGAGACTATAGCCTGGAGGTCTGTGT\nGGCATTCCATCATGATTGCCTCAAAGACTAGGGATGTTTCCATGAATGGAGTATTTTTTTGTTATTAAAA\nATTTCTGAACTGTTACTCCCAAATTTCTCTGAACAACTTTTGAAGCTTTTCATATGCCTCCTATAGCATA\nTGTTGGGGTAGATAGTTCCATGAAGTATGTACACTCTATAGATATAAAGAAAGAGGTTCTTTTCTTTCTC\nTCAGACTTACATTTCCACATGGGAATTGGCACAGGTGGGGAGTAGGTGAAAGAGCCCAGCAGGCTGAATG\nCCTTCAACAATCATTTTACCACGTGGTAAATGTGGTACTTACTCTCTGCTACCTCATATATGTCACCTCG\nCTTATGATCAAATAAAATGGGCATGTAGATATGCTTTATGAATAGTAAAAACATGAATGTCAACTTTTTT\nTAA

In [7]:
id_proteina = pesquisa_ncbi(email,gene,db='protein',retmax=1, display='n').split()[1]
id_proteina

Encontrados 57 resultados em protein. Irá ser processado 1 resultado.



'DQB2_HUMAN'

In [8]:
from Bio import SeqIO
from Bio import ExPASy

fich_fasta = 'gene_search() output\\' + gene + '.fasta'

# Retiramos a sequência de DNA a partir do ficheiro FASTA

seq = SeqIO.read(open(fich_fasta),format='fasta')

In [9]:
# Procedemos à sua transcrição e tradução para obter a sequência de aminoácidos

seq_trans = seq.seq.transcribe()
seq_trad  = seq_trans.translate()

In [10]:
# Procedemos à contagem de aminoácidos:
# Primeiro para a sequência traduzida e depois para o conjunto de sequências após a remoção de codões stop

from collections import Counter

contagem_amino_total = Counter(seq_trad)


In [11]:
handle = ExPASy.get_sprot_raw(id_proteina)
seq_record = SeqIO.read(handle, "swiss")
id = seq_record.id
seq = seq_record.seq
tam = len(seq_record.seq)
name = seq_record.name
desc = seq_record.description
com = seq_record.annotations["comment"]
taxon = seq_record.annotations["taxonomy"]
organism = seq_record.annotations["organism"]
key = seq_record.annotations["keywords"]
print(f"ID {id} \n Sequência: {seq} \n Tamanho da sequência: {tam} bp")
print(f"Nome: {name} \n Descrição: {desc} \n Taxonomia: {taxon} \n Organismo: {organism} \n Keywords: {key}")

ID P05538 
 Sequência: MSWKMALQIPGGFWAAAVTVMLVMLSTPVAEARDFPKDFLVQFKGMCYFTNGTERVRGVARYIYNREEYGRFDSDVGEFQAVTELGRSIEDWNNYKDFLEQERAAVDKVCRHNYEAELRTTLQRQVEPTVTISPSRTEALNHHNLLVCSVTDFYPAQIKVRWFRNDQEETAGVVSTSLIRNGDWTFQILVMLEITPQRGDIYTCQVEHPSLQSPITVEWRAQSESAQSKMLSGIGGFVLGLIFLGLGLIIRHRGQKGPRGPPPAGLLH 
 Tamanho da sequência: 268 bp
Nome: DQB2_HUMAN 
 Descrição: RecName: Full=HLA class II histocompatibility antigen, DQ beta 2 chain; AltName: Full=HLA class II histocompatibility antigen, DX beta chain; AltName: Full=MHC class II antigen DQB2; Flags: Precursor; 
 Taxonomia: ['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo'] 
 Organismo: Homo sapiens (Human) 
 Keywords: ['Adaptive immunity', 'Alternative splicing', 'Cell membrane', 'Disulfide bond', 'Endoplasmic reticulum', 'Endosome', 'Glycoprotein', 'Golgi apparatus', 'Immunity', 'Lysosome', 'Membrane', 'MHC II', 'Re